In [1]:
import multiprocessing
import numpy as np
import multiprocess as mp
import os
import fsspec
import pandas as pd
import xarray as xr
from tqdm import tqdm
import geopandas
import pandas as pd
import datetime as dt
import time as time

In [2]:
pnwNP = pd.read_csv("../data/pnwNP_Info.csv")

### Pull out sites
shp = geopandas.read_file("../data/VIC_UW/shapefiles/columbia_seg.shp")
shp = pd.merge(pnwNP,shp,left_on='comid',right_on='POI_ID').fillna(0)

In [3]:
fs = fsspec.filesystem('s3', anon=True)
_file = fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr')

ds = xr.open_dataset(fs.get_mapper(_file[0]), engine='zarr', backend_kwargs={'consolidated': True})

In [12]:
for i in range(len(shp)):
    print(shp['comid'][i])
    ## Get NWM 2.1 data
    # slice all data using a specific reach identifier and time range
    dat = pd.read_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")
    dat['time'] = pd.to_datetime(dat['time'])
    timerange = slice(dat['time'][0], dat['time'].iloc[-1])
    df = ds.sel(feature_id=shp['comid'][i],
                 time=timerange).streamflow.persist() 

    NWM = pd.DataFrame(df.to_pandas()).resample('1d').mean()
    NWM['time'] = pd.to_datetime(NWM.index)
    NWM.columns = ["streamflow_NWM2d1","time"]
    NWM = NWM.reset_index(drop=True)
    NWM["gage"] = shp['gage'][i]



    ## Combine with previous modeled data
    datMain = pd.merge(dat,NWM, on='time',how='inner')

    datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+"wNWM2d1.csv")

24538136
24537944
24293810
24397698
24416110
24471640
23148386
24478889
24478889
23184823
23223074
23238296
23238318
23238314
23250977
23252011
23275840
23320100
23413305
23412937
24158991
23481707
23637966
23637700
23659590
23659572
24249014
23935979


In [10]:
def NWMmp(i):
    print(shp['comid'][i])
    ## Get NWM 2.1 data
    # slice all data using a specific reach identifier and time range
    dat = pd.read_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")
    dat['time'] = pd.to_datetime(dat['time'])
    timerange = slice(dat['time'][0], dat['time'].iloc[-1])
    df = ds.sel(feature_id=shp['comid'][i],
                 time=timerange).streamflow.persist() 

    NWM = pd.DataFrame(df.to_pandas()).resample('1d').mean()
    NWM['time'] = pd.to_datetime(NWM.index)
    NWM.columns = ["streamflow_NWM2d1","time"]
    NWM = NWM.reset_index(drop=True)
    NWM["gage"] = shp['gage'][i]



    ## Combine with previous modeled data
    datMain = pd.merge(dat,NWM, on='time',how='inner')

    datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+"wNWM2d1.csv")
    datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+"wNWM2d1.csv")
    print(time.process_time())



In [11]:
import numpy as np
import multiprocess as mp
import os


if __name__ == '__main__':
    N = mp.cpu_count()

    with mp.Pool(processes = N) as p:
        p.map(NWMmp, list(range(len(shp))))
        p.close()

2453794424538136243976982314838624293810244161102447164024478889







2447888923184823232383182322307423238314232382962325097723252011







2327584023320100241589912348170723412937234133052363796623637700







23659590236595722424901423935979





RuntimeError: This class is not fork-safe

In [ ]:
# This step takes a bit longer because it's actually returning the data
dat = dat.resample(time='1d').mean()

NWM = pd.DataFrame(dat.to_pandas())
NWM['time'] = pd.to_datetime(NWM.index)
NWM.columns = ["streamflow_NWM2d1","time"]
NWM = NWM.reset_index(drop=True)
NWM["gage"] = shp['gage'][i]


In [ ]:
! pip3 install multiprocess

In [ ]:
import numpy as np
import multiprocess as mp
import os


if __name__ == '__main__':
    N= mp.cpu_count()
    
    def NWM(i):
        print(shp['comid'][i])
    #     ## Get NWM 2.1 data
    #     # slice all data using a specific reach identifier and time range
    #     dat = pd.read_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+".csv")
    #     dat['time'] = pd.to_datetime(dat['time'])
    #     timerange = slice(dat['time'][0], dat['time'].iloc[-1])
    #     dat = ds.sel(feature_id=shp['comid'][i],
    #                  time=timerange).streamflow.persist() 

    #     NWM = pd.DataFrame(dat.to_pandas()).resample('1d').mean()
    #     NWM['time'] = pd.to_datetime(NWM.index)
    #     NWM.columns = ["streamflow_NWM2d1","time"]
    #     NWM = NWM.reset_index(drop=True)
    #     NWM["gage"] = shp['gage'][i]



    #     ## Combine with previous modeled data
    #     datMain = pd.merge(dat,NWM, on='time',how='inner')
        datMain = pd.DataFrame([0,1,2])
        datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+"wNWM2d1.csv")

    with mp.Pool(processes = N) as p:
        p.map(NWM, list(range(len(shp))))

In [ ]:
## Combine with previous modeled data
datMain = pd.merge(dat,NWM, on='time',how='inner')

datMain.to_csv('../data/pnwNP_modeledData/'+str(shp['gage'][i])+"wNWM2d1.csv")

In [ ]:
# timerange = slice('2011-01-31', '2011-02-28')
# dat = ds.sel(feature_id=comid,time=timerange).streamflow.persist() 
dat = ds.sel(feature_id=comid).streamflow.persist() 

In [ ]:
dat = dat.resample(time='1d').mean()

In [ ]:
tetNWM = pd.DataFrame(dat.to_pandas())

In [ ]:
NWM = tetNWM

In [ ]:
NWM = pd.DataFrame(dat.to_pandas())
NWM['time'] = pd.to_datetime(NWM.index)
NWM.columns = ["streamflow_NWM2d1","time"]
NWM = NWM.reset_index(drop=True)
# NWM["gage"] = shp['gage'][i]
NWM